In [8]:
versioninfo()

Julia Version 0.6.4
Commit 9d11f62bcb (2018-07-09 19:09 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-4500U CPU @ 1.80GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell MAX_THREADS=16)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


In [15]:
using StatsBase
using IterTools

In [16]:
function evaluate(solution, distances)
    objective_function = 0
    
    for i in 1:(size(solution)[1]-1)
        objective_function += distances[solution[i],solution[i+1]]
    end

    objective_function += distances[solution[end],solution[1]]
    
    return objective_function
end

function first_sequential_k_opt2(solution, near_cities, distances)
    #solution_ref = copy(solution)
    #push!(solution_ref, solution[end])
    
    #search
    for i in 1:length(solution)-1
        used_cities = []
        old_weights = distances[solution[i],solution[i+1]]
        new_gain = 0
        
        new_solution = vcat(solution[i:-1:1], solution[end:-1:i+1])
        
        push!(used_cities, solution[i])
        push!(used_cities, solution[i+1])
        for k = 2:5
            found = false
            for city in near_cities[new_solution[end]]
                candidate = findin(new_solution, city)
                if length(candidate) == 1
                    j = candidate[1]
                    if ~(city in used_cities) & ~(new_solution[j+1] in used_cities)
                        if ((old_weights + distances[new_solution[j],new_solution[j+1]]) >
                            (new_gain + distances[new_solution[end], new_solution[j]]
                            + distances[new_solution[1], new_solution[j+1]]))
                            
                            found = true
                            
                            old_weights += distances[new_solution[j],new_solution[j+1]]
                            new_gain += distances[new_solution[end],new_solution[j]]
                            
                            push!(used_cities, new_solution[j])
                            push!(used_cities, new_solution[j+1])
                            
                            new_solution = vcat(new_solution[1:j], new_solution[end:-1:j+1])
                            
                            break
                        end

                    end
                end
            end
            if found == false
                break
            end
        end
        if length(used_cities) > 2
            return new_solution, true
        end 
    end
    #@show solution
    return solution, false 
end

first_sequential_k_opt2 (generic function with 1 method)

In [17]:
function TSP(distances)
    
    n_cities = size(distances)[1]
    max_near_cities = min(5,n_cities-1)

    near_cities = Dict()

    # STEP 1 - preprocessing nearest cities for heuristic

    for i in 1:n_cities
        near_cities[i] = selectperm(distances[i, :], 1:max_near_cities+1)
        shift!(near_cities[i])
    end

    # STEP 2 - UPPERBOUND (quasi) Lin Kernighan - First improvement sequential 5-opt
    begin
        upper_bound = 1e10
        a = StatsBase.sample(1:n_cities, n_cities, replace=false, ordered=false)
        best_heuristic_solution = []

        for i in 1:100*(n_cities)
            a, better  = first_sequential_k_opt2(a, near_cities, distances)
            if better == false
                value = evaluate(a, distances)
                if value < upper_bound
                    upper_bound = value
                    best_heuristic_solution = copy(a)
                end
                a = StatsBase.sample(1:n_cities, n_cities, replace=false, ordered=false)
            end
        end

        value = evaluate(a, distances)
        if value < upper_bound
            upper_bound = value
        end
    end

    return upper_bound, best_heuristic_solution
end

TSP (generic function with 1 method)

In [18]:
centers = readdlm("routing_files/centers.csv",',')
clusters = readdlm("routing_files/clusters.csv",',')
euclidean_distances = readdlm("routing_files/euclidean_distances.csv",',')
coord_of_POI = readdlm("routing_files/coord_of_POI.csv",',');

In [19]:
base_coord = coord_of_POI[1,:];
relative_coord = (coord_of_POI[2:end,:] .- base_coord');
n_POI = size(relative_coord)[1]

142

In [20]:
@time open("routing_files/route_order.txt", "w") do f

    for i in 1:Int(maximum(clusters[:,1]))

        idx_cluster = collect(1:n_POI)[(clusters .== i)[:,1]]
        idx_cluster = vcat([1], idx_cluster .+ 1)

        distance, order = TSP(euclidean_distances[idx_cluster,idx_cluster])
        idx_cluster[order]
        pos = find(idx_cluster[order] .== 1)[1]
        idx_order = copy(idx_cluster[order])
        if pos > 1
            idx_order = vcat(idx_order[pos:end], idx_order[1:pos-1])
        end
        
        
        write(f, string(coord_of_POI[idx_order,:],"\n"))
    end

end

  0.774311 seconds (4.34 M allocations: 295.339 MiB, 5.86% gc time)
